Сегодня мы узнаем, как можно относительно быстро получить список отзывов к определенному объекту на Яндекс.Карты.

Для начала работы нам необходимо подгрузить модули для работы с сохраненной страницей в формате html.


In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

Далее нам нужно загрузить наш файл с сохраненной страницей в формате html, изменив путь в строке.

In [63]:
with open('/content/Зимний театр — Яндекс Карты.html') as f:
    content = f.read()

Включаем в работу модуль для обработки для парсинга html-страниц BeautifulSoup.

In [64]:
soup = BeautifulSoup(content, features='html.parser')

Текст

In [7]:
#google_comment_class = "wiI7pd"
#google_post_time_class = 'rsqaWe'
#google_rating_class = 'kvMYJc'
#google_author_class = 'd4r55'
#google_author_status_class = 'RfnDt'

Функция для извлечения комментариев для Яндекс.Карт

In [94]:
def get_reviews_from_YandexMaps(bs4_soup = soup):

    reviews = []
    authors = []
    authors_status = []
    authors_ratings = []
    publication_date = []

    for result in soup.findAll('div', class_='business-reviews-card-view__review'):
        try:
            reviews.append(result.find('span', class_='business-review-view__body-text').get_text())
        except:
            reviews.append(np.NaN)

        try:
            authors.append(result.find('div', class_='business-review-view__author').\
                           find('span', attrs={'itemprop': 'name'}).get_text())
        except:
            authors.append(np.NaN)
            
        try:
            authors_status.append(result.find('div', class_='business-review-view__author').\
                                  find('div', class_='business-review-view__author-profession').get_text())
        except:
            authors_status.append(np.NaN)
        
        try:
            authors_ratings.append(result.find('div', class_='business-review-view__rating').\
                                       find('span', attrs={'itemprop': 'reviewRating'}).\
                                       find('meta', attrs={'itemprop': 'ratingValue'})['content'])
        except:
            authors_ratings.append(np.NaN)
        
        try:
            publication_date.append(result.find('span', class_='business-review-view__date').\
                                    find('meta')['content'])
        except:
            publication_date.append(0)
    
    authors_ratings_int = [float(n) for n in authors_ratings]

    data = pd.DataFrame.from_dict({'author': authors,\
                               'status': authors_status,\
                               'rating': authors_ratings_int,\
                               'publication_date': publication_date,\
                               'review': reviews})
    data['publication_date'] = pd.to_datetime(data['publication_date']).dt.date

    return data


In [95]:
data = get_reviews_from_YandexMaps(bs4_soup=soup)

In [96]:
data.head(5)

,author,status,rating,publication_date,review
0,вера ц.,Знаток города 4 уровня,3.0,2022-11-08,"Зимний театр, это отличная атмосфера, стиль и ..."
1,юлия кузьмич,Знаток города 3 уровня,5.0,2022-07-13,"Потрясающая атмосфера, шикарный интерьер, удач..."
2,Комиссарова Анна,Знаток города 7 уровня,5.0,2022-08-03,"Зимний театр - это очень прекрасный театр, где..."
3,вера ц.,Знаток города 4 уровня,3.0,2022-11-08,"Зимний театр, это отличная атмосфера, стиль и ..."
4,юлия кузьмич,Знаток города 3 уровня,5.0,2022-07-13,"Потрясающая атмосфера, шикарный интерьер, удач..."


In [97]:
data.to_excel('result.xlsx')